# Herschel Stripe 82 VICS82 - VHS merging

VICS 82 and VHS contain VISTA J and Ks fluxes which need to be merged

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
0246c5d (Thu Jan 25 17:01:47 2018 +0000) [with local modifications]
This notebook was executed on: 
2018-02-20 18:50:49.783850


In [26]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time
from collections import OrderedDict

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

In [3]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [4]:
vics82 = Table.read("{}/VICS82.fits".format(TMP_DIR))
vhs = Table.read("{}/VISTA-VHS.fits".format(TMP_DIR))

## II - Merging tables

We first merge the optical catalogues and then add the infrared ones:  VHS, VICS82, 

At every step, we look at the distribution of the distances to the nearest source in the merged catalogue to determine the best crossmatching radius.

### VHS

In [5]:
master_catalogue = vhs
master_catalogue['vhs_ra'].name = 'ra'
master_catalogue['vhs_dec'].name = 'dec'


## Add VICS82

In [6]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(vics82['vics82_ra'], vics82['vics82_dec'])
)

HELP Warning: There weren't any cross matches. The two surveys probably don't overlap.


In [7]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, vics82, "vics82_ra", "vics82_dec", radius=0.8*u.arcsec)


### Cleaning

When we merge the catalogues, astropy masks the non-existent values (e.g. when a row comes only from a catalogue and has no counterparts in the other, the columns from the latest are masked for that row). We indicate to use NaN for masked values for floats columns, False for flag columns and -1 for ID columns.

In [8]:
for col in master_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        master_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        master_catalogue[col].fill_value = 0
    elif "id" in col:
        master_catalogue[col].fill_value = -1
        
master_catalogue = master_catalogue.filled()

In [9]:
master_catalogue[:10].show_in_notebook()

idx,vhs_id,ra,dec,vhs_stellarity,m_vhs_y,merr_vhs_y,m_ap_vhs_y,merr_ap_vhs_y,m_vhs_j,merr_vhs_j,m_ap_vhs_j,merr_ap_vhs_j,m_vhs_h,merr_vhs_h,m_ap_vhs_h,merr_ap_vhs_h,m_vhs_k,merr_vhs_k,m_ap_vhs_k,merr_ap_vhs_k,f_vhs_y,ferr_vhs_y,flag_vhs_y,f_ap_vhs_y,ferr_ap_vhs_y,f_vhs_j,ferr_vhs_j,flag_vhs_j,f_ap_vhs_j,ferr_ap_vhs_j,f_vhs_h,ferr_vhs_h,flag_vhs_h,f_ap_vhs_h,ferr_ap_vhs_h,f_vhs_k,ferr_vhs_k,flag_vhs_k,f_ap_vhs_k,ferr_ap_vhs_k,vhs_flag_cleaned,vhs_flag_gaia,flag_merged,vics82_id,vics82_stellarity,vics82_flag_cleaned,f_wircam_j,f_vista_j,ferr_wircam_j,ferr_vista_j,m_wircam_j,m_vista_j,merr_wircam_j,merr_vista_j,flag_wircam_j,flag_vista_j,f_ap_wircam_j,f_ap_vista_j,ferr_ap_wircam_j,ferr_ap_vista_j,m_ap_wircam_j,m_ap_vista_j,merr_ap_wircam_j,merr_ap_vista_j,f_wircam_ks,f_vista_ks,ferr_wircam_ks,ferr_vista_ks,m_wircam_ks,m_vista_ks,merr_wircam_ks,merr_vista_ks,flag_wircam_ks,flag_vista_ks,f_ap_wircam_ks,f_ap_vista_ks,ferr_ap_wircam_ks,ferr_ap_vista_ks,m_ap_wircam_ks,m_ap_vista_ks,merr_ap_wircam_ks,merr_ap_vista_ks,vics82_flag_gaia
,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,472448751411,0.781408839858,0.0890078729011,0.999657,nan,nan,nan,nan,10.6256,0.000379132,11.0127,0.000459667,10.2142,0.000453802,10.5483,0.000520294,10.0804,0.000651023,10.2967,0.000605307,nan,nan,False,nan,nan,204058.0,71.2559,False,142869.0,60.4863,298085.0,124.59,False,219116.0,105.002,337159.0,202.166,False,276273.0,154.024,False,3,False,-1,nan,False,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,0
1,472448749614,0.143494283104,0.174711331143,0.999657,nan,nan,nan,nan,10.6885,0.000386127,11.014,0.000463202,10.1433,0.00046483,10.4889,0.000548676,10.1333,0.000672656,10.3985,0.000638829,nan,nan,False,nan,nan,192582.0,68.4889,False,142689.0,60.8748,318178.0,136.219,False,231430.0,116.953,321125.0,198.949,False,251531.0,147.997,False,3,False,-1,nan,False,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,0
2,472471300934,0.0496896315432,-0.360734598751,0.9,nan,nan,nan,nan,nan,nan,nan,nan,9.50416,0.000348363,10.9399,0.00055988,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,573237.0,183.925,False,152766.0,78.7765,nan,nan,False,nan,nan,False,0,False,-1,nan,False,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,0
3,472471300942,0.0481301624509,-0.361356272804,0.9,nan,nan,nan,nan,nan,nan,nan,nan,9.29619,0.000309161,10.9588,0.000567745,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,694263.0,197.69,False,150130.0,78.505,nan,nan,False,nan,nan,False,0,False,-1,nan,False,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,0
4,472471275977,0.292573308208,-0.0763851838315,0.944606,nan,nan,nan,nan,9.74752,0.000326092,12.8316,0.0010375,10.4151,0.000434455,11.003,0.000570257,9.23457,0.000457213,10.5647,0.000651892,nan,nan,False,nan,nan,458132.0,137.596,False,26753.4,25.5647,247723.0,99.1259,False,144145.0,75.7089,734804.0,309.432,False,215824.0,129.584,False,3,False,-1,nan,False,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,0
5,472471300019,0.291958731973,-0.0763485415078,0.9,nan,nan,nan,nan,nan,nan,nan,nan,10.9348,0.000587732,11.0332,0.000578617,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,153483.0,83.0836,False,140193.0,74.7123,nan,nan,False,nan,nan,False,1,False,-1,nan,False,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,0
6,472448745265,0.860644844416,0.349445245846,0.999657,nan,n

## III - Merging flags and stellarity

Each pristine catalogue contains a flag indicating if the source was associated to a another nearby source that was removed during the cleaning process.  We merge these flags in a single one.

In [10]:
flag_cleaned_columns = [column for column in master_catalogue.colnames
                        if 'flag_cleaned' in column]

flag_column = np.zeros(len(master_catalogue), dtype=bool)
for column in flag_cleaned_columns:
    flag_column |= master_catalogue[column]
    
master_catalogue.add_column(Column(data=flag_column, name="vista_flag_cleaned"))
master_catalogue.remove_columns(flag_cleaned_columns)

Each pristine catalogue contains a flag indicating the probability of a source being a Gaia object (0: not a Gaia object, 1: possibly, 2: probably, 3: definitely).  We merge these flags taking the highest value.

In [11]:
flag_gaia_columns = [column for column in master_catalogue.colnames
                     if 'flag_gaia' in column]

master_catalogue.add_column(Column(
    data=np.max([master_catalogue[column] for column in flag_gaia_columns], axis=0),
    name="vista_flag_gaia"
))
master_catalogue.remove_columns(flag_gaia_columns)

Each prisitine catalogue may contain one or several stellarity columns indicating the probability (0 to 1) of each source being a star.  We merge these columns taking the highest value.

In [12]:
stellarity_columns = [column for column in master_catalogue.colnames
                      if 'stellarity' in column]

master_catalogue.add_column(Column(
    data=np.nanmax([master_catalogue[column] for column in stellarity_columns], axis=0),
    name="vista_stellarity"
))
master_catalogue.remove_columns(stellarity_columns)

## VIII - Cross-identification table

We are producing a table associating to each HELP identifier, the identifiers of the sources in the pristine catalogue. This can be used to easily get additional information from them.

In [13]:
master_catalogue.add_column(Column(data=np.arange(len(master_catalogue)), 
                              name="vista_intid"))


In [14]:

id_names = []
for col in master_catalogue.colnames:
    if '_id' in col:
        id_names += [col]
    if '_intid' in col:
        id_names += [col]
        
print(id_names)

['vhs_id', 'vics82_id', 'vista_intid']


## VI - Choosing between multiple values for the same filter



In [15]:
vista_origin = Table()
vista_origin.add_column(master_catalogue['vista_intid'])

In [16]:
vista_stats = Table()
vista_stats.add_column(Column(data=['j','ks'], name="Band"))
for col in ["VHS", "VICS82"]:
    vista_stats.add_column(Column(data=np.full(2, 0), name="{}".format(col)))
    vista_stats.add_column(Column(data=np.full(2, 0), name="use {}".format(col)))
    vista_stats.add_column(Column(data=np.full(2, 0), name="{} ap".format(col)))
    vista_stats.add_column(Column(data=np.full(2, 0), name="use {} ap".format(col)))

In [17]:
vista_bands = ['j','ks'] # Lowercase naming convention (k is Ks)
for band in vista_bands:


    # VISTA total flux 
    has_vhs = ~np.isnan(master_catalogue['f_vhs_' + band.strip('s')])
    has_vics82 = ~np.isnan(master_catalogue['f_vista_' + band])

    use_vhs = has_vhs 
    use_vics82 = has_vics82 & ~has_vhs


    master_catalogue['f_vista_' + band][use_vhs] = master_catalogue['f_vhs_' + band.strip('s')][use_vhs]
    master_catalogue['ferr_vista_' + band][use_vhs] = master_catalogue['ferr_vhs_' + band.strip('s')][use_vhs]
    master_catalogue['m_vista_' + band][use_vhs] = master_catalogue['m_vhs_' + band.strip('s')][use_vhs]
    master_catalogue['merr_vista_' + band][use_vhs] = master_catalogue['merr_vhs_' + band.strip('s')][use_vhs]
    master_catalogue['flag_vista_' + band][use_vhs] = master_catalogue['flag_vhs_' + band.strip('s')][use_vhs]

    master_catalogue.remove_columns(['f_vhs_' + band.strip('s'),
                               'ferr_vhs_' + band.strip('s'),
                               'm_vhs_' + band.strip('s'), 
                               'merr_vhs_' + band.strip('s'),
                               'flag_vhs_' + band.strip('s')])

    origin = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin[use_vhs] = "VHS"
    origin[use_vics82] = "VICS82"

    
    vista_origin.add_column(Column(data=origin, name= 'f_vista_' + band ))
    
    # VISTA aperture flux 
    has_ap_vhs = ~np.isnan(master_catalogue['f_ap_vhs_' + band.strip('s')])
    has_ap_vics82 = ~np.isnan(master_catalogue['f_ap_vista_' + band])

    use_ap_vhs = has_ap_vhs 
    use_ap_vics82 = has_ap_vics82 & ~has_ap_vhs


    master_catalogue['f_ap_vista_' + band][use_ap_vhs] = master_catalogue['f_ap_vhs_' + band.strip('s')][use_ap_vhs]
    master_catalogue['ferr_ap_vista_' + band][use_ap_vhs] = master_catalogue['ferr_ap_vhs_' + band.strip('s')][use_ap_vhs]
    master_catalogue['m_ap_vista_' + band][use_ap_vhs] = master_catalogue['m_ap_vhs_' + band.strip('s')][use_ap_vhs]
    master_catalogue['merr_ap_vista_' + band][use_ap_vhs] = master_catalogue['merr_ap_vhs_' + band.strip('s')][use_ap_vhs]
  

    master_catalogue.remove_columns(['f_ap_vhs_' + band.strip('s'),
                               'ferr_ap_vhs_' + band.strip('s'),
                               'm_ap_vhs_' + band.strip('s'), 
                               'merr_ap_vhs_' + band.strip('s')])

    origin_ap = np.full(len(master_catalogue), '     ', dtype='<U5')
    origin_ap[use_ap_vhs] = "VHS"
    origin_ap[use_ap_vics82] = "VICS82"

    
    vista_origin.add_column(Column(data=origin_ap, name= 'f_ap_vista_' + band ))   

   
    vista_stats['VHS'][vista_stats['Band'] == band] = np.sum(has_vhs)
    vista_stats['VICS82'][vista_stats['Band'] == band] = np.sum(has_vics82)

    vista_stats['use VHS'][vista_stats['Band'] == band] = np.sum(use_vhs)
    vista_stats['use VICS82'][vista_stats['Band'] == band] = np.sum(use_vics82)

    vista_stats['VHS ap'][vista_stats['Band'] == band] = np.sum(has_ap_vhs)
    vista_stats['VICS82 ap'][vista_stats['Band'] == band] = np.sum(has_ap_vics82)

    vista_stats['use VHS ap'][vista_stats['Band'] == band] = np.sum(use_ap_vhs)
    vista_stats['use VICS82 ap'][vista_stats['Band'] == band] = np.sum(use_ap_vics82)


In [18]:
vista_stats.show_in_notebook()

idx,Band,VHS,use VHS,VHS ap,use VHS ap,VICS82,use VICS82,VICS82 ap,use VICS82 ap
0,j,43636,43636,43633,43633,34934,34934,34900,34900
1,ks,35049,35049,35042,35042,41273,41273,41094,41094


In [19]:
vista_origin.write("{}/herschel-stripe-82_vista_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

## Rename old vhs cols

In [27]:

renaming = OrderedDict({
    '_vhs_y': '_vista_y',
    '_vhs_h': '_vista_h',

})


for col in master_catalogue.colnames:
    for rename_col in list(renaming):
        if rename_col in col:
            master_catalogue.rename_column(col, col.replace(rename_col, renaming[rename_col]))

## IX - Saving the catalogue

In [28]:
master_catalogue["ra"].name = "vista_ra"
master_catalogue["dec"].name = "vista_dec"
master_catalogue["flag_merged"].name = "vista_flag_merged"

KeyError: 'ra'

In [29]:
columns = ["vhs_id", "vics82_id", "vista_intid",
           'vista_ra', 'vista_dec', 'vista_flag_merged',  
           'vista_flag_cleaned',  'vista_flag_gaia', 'vista_stellarity']

bands = [column[5:] for column in master_catalogue.colnames if 'f_ap' in column]
for band in bands:
    columns += ["f_ap_{}".format(band), "ferr_ap_{}".format(band),
                "m_ap_{}".format(band), "merr_ap_{}".format(band),
                "f_{}".format(band), "ferr_{}".format(band),
                "m_{}".format(band), "merr_{}".format(band),
                "flag_{}".format(band)]    


In [30]:
# We check for columns in the master catalogue that we will not save to disk.
print("Missing columns: {}".format(set(master_catalogue.colnames) - set(columns)))

Missing columns: set()


In [31]:
master_catalogue[:10].show_in_notebook()

idx,vhs_id,vista_ra,vista_dec,m_vista_y,merr_vista_y,m_ap_vista_y,merr_ap_vista_y,m_vista_h,merr_vista_h,m_ap_vista_h,merr_ap_vista_h,f_vista_y,ferr_vista_y,flag_vista_y,f_ap_vista_y,ferr_ap_vista_y,f_vista_h,ferr_vista_h,flag_vista_h,f_ap_vista_h,ferr_ap_vista_h,vista_flag_merged,vics82_id,f_wircam_j,f_vista_j,ferr_wircam_j,ferr_vista_j,m_wircam_j,m_vista_j,merr_wircam_j,merr_vista_j,flag_wircam_j,flag_vista_j,f_ap_wircam_j,f_ap_vista_j,ferr_ap_wircam_j,ferr_ap_vista_j,m_ap_wircam_j,m_ap_vista_j,merr_ap_wircam_j,merr_ap_vista_j,f_wircam_ks,f_vista_ks,ferr_wircam_ks,ferr_vista_ks,m_wircam_ks,m_vista_ks,merr_wircam_ks,merr_vista_ks,flag_wircam_ks,flag_vista_ks,f_ap_wircam_ks,f_ap_vista_ks,ferr_ap_wircam_ks,ferr_ap_vista_ks,m_ap_wircam_ks,m_ap_vista_ks,merr_ap_wircam_ks,merr_ap_vista_ks,vista_flag_cleaned,vista_flag_gaia,vista_stellarity,vista_intid
,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,472448751411,0.781408839858,0.0890078729011,nan,nan,nan,nan,10.2142,0.000453802,10.5483,0.000520294,nan,nan,False,nan,nan,298085.0,124.59,False,219116.0,105.002,False,-1,nan,204058.4375,nan,71.2558517456,nan,10.6256132126,nan,0.00037913187407,False,False,nan,142869.0,nan,60.486289978,nan,11.0127,nan,0.000459667207906,nan,337159.25,nan,202.165557861,nan,10.080411911,nan,0.000651023117825,False,False,nan,276273.0,nan,154.024353027,nan,10.2967,nan,0.000605307111982,False,3,0.999657213688,0
1,472448749614,0.143494283104,0.174711331143,nan,nan,nan,nan,10.1433,0.00046483,10.4889,0.000548676,nan,nan,False,nan,nan,318178.0,136.219,False,231430.0,116.953,False,-1,nan,192581.65625,nan,68.4889373779,nan,10.6884622574,nan,0.000386126688682,False,False,nan,142689.0,nan,60.8748054504,nan,11.014,nan,0.000463202042738,nan,321124.75,nan,198.949264526,nan,10.1333150864,nan,0.000672655762173,False,False,nan,251531.0,nan,147.996856689,nan,10.3985,nan,0.000638829253148,False,3,0.999657213688,1
2,472471300934,0.0496896315432,-0.360734598751,nan,nan,nan,nan,9.50416,0.000348363,10.9399,0.00055988,nan,nan,False,nan,nan,573237.0,183.925,False,152766.0,78.7765,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,False,0,0.899999976158,2
3,472471300942,0.0481301624509,-0.361356272804,nan,nan,nan,nan,9.29619,0.000309161,10.9588,0.000567745,nan,nan,False,nan,nan,694263.0,197.69,False,150130.0,78.505,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,False,0,0.899999976158,3
4,472471275977,0.292573308208,-0.0763851838315,nan,nan,nan,nan,10.4151,0.000434455,11.003,0.000570257,nan,nan,False,nan,nan,247723.0,99.1259,False,144145.0,75.7089,False,-1,nan,458131.5,nan,137.595962524,nan,9.74752426147,nan,0.000326091801981,False,False,nan,26753.4,nan,25.5647392273,nan,12.8316,nan,0.001037496957,nan,734804.1875,nan,309.432281494,nan,9.23457050323,nan,0.000457212707261,False,False,nan,215824.0,nan,129.583984375,nan,10.5647,nan,0.000651891517919,False,3,0.944606423378,4
5,472471300019,0.291958731973,-0.0763485415078,nan,nan,nan,nan,10.9348,0.000587732,11.0332,0.000578617,nan,nan,False,nan,nan,153483.0,83.0836,False,140193.0,74.7123,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,nan,nan,nan,nan,nan,nan,False,1,0.899999976158,5
6,472448745265,0.860644844416,0.349445245846,nan,nan,nan,nan,10.2137,0.000431752,10.8897,0.0005881,nan,nan,False,nan,nan,298203.0,118.583,False,159995.0,86.6629,False,-1,nan,117336.515625,nan,54.0334510803,nan,11.2264165878,nan,0.000499981397297,False,False,nan,101973.0,nan,50.620716095,nan,11.3788,nan,0.000538974127267,nan,221259.5625,nan,175.496902466,nan,10.5377445221,nan,0.000861175591126,False,False,nan,210959.0,nan,136.306625366,nan,10.5895,nan,0.000701524375472,False,3,0.999657213688,6
7,472448749010,0.115629264879,0.2050312

In [32]:
master_catalogue[columns].write("{}/vista_merged_catalogue_herschel-stripe-82.fits".format(TMP_DIR), overwrite=True)